In [33]:
import numpy as np
import pandas as pd
import math
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn import preprocessing
# from src2.model import *
pd.options.display.max_columns = 100


train = pd.read_csv('../data/train_dataset.csv')
test = pd.read_csv('../data/test_dataset.csv')
target = train['信用分']
data = pd.concat([train,test],axis=0,ignore_index=True)
data0 = data.copy()
data['信用分'] = data['信用分'].fillna(-1)



def feature_count(data, features):
    if len(set(data[features])) == len(data[features]):
        print('equal feature')
        return data
    new_feature = 'count'
    new_feature += '_' + features
    temp = data.groupby(features).size().reset_index().rename(columns={0: new_feature})
    data = data.merge(temp, 'left', on=features)
    return data

def endnumber(x):
    x_2_str = str(x)
    res = x_2_str.split(".")
    return int(res[-1])

def freq_coding(x, countdict):
    if (countdict[x] > 0) and (countdict[x] <= 50):
        return 0
    elif (countdict[x] > 50) and (countdict[x] <= 300):
        return 1
    else:
        return 2
    
def num_ceilling(x):
    return float('%.3f' % (x/math.ceil(x) if int(x) != 0 else 0) )
    
def omit_value(x):
    if x <= 500:
        return 0
    elif (x>500) and (x<1000):
        return 1
    else:
        return 2

def standard(x):
    return preprocessing.StandardScaler().fit_transform(x.values.reshape(-1,1))

# def log(func):
#     def wrapper():
#         print("输入时data的列数:%s" % self.data.shape[1])
#         return func(data)
#     return wrapper

/home/wrj/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


单一衍生：数据分布、频数编码、feature_count、endword、业务推导     
复合衍生：数据分布、业务推导

In [34]:
predictors = [i for i in data.columns if i not in ['用户编码','信用分','userno']]
predictors_category = ['用户实名制是否通过核实','是否大学生客户','是否黑名单客户','是否4G不健康客户',
                       '缴费用户当前是否欠费缴费','是否经常逛商场的人','当月是否逛过福州仓山万达',
                       '当月是否到过福州山姆会员店','当月是否看电影','当月是否景点游览','当月是否体育场馆消费',
                       '用户最近一次缴费距今时长（月）']
predictors_identity = ['用户年龄','近三个月月均商场出现次数']
predictors_consume = ['用户账单当月总费用（元）','缴费用户最近一次缴费金额（元）','用户近6个月平均消费值（元）',
                      '用户当月账户余额（元）','用户话费敏感度','当月通话交往圈人数','用户网龄（月）']
predictors_apps = ['当月网购类应用使用次数','当月物流快递类应用使用次数','当月金融理财类应用使用总次数',
                   '当月视频播放类应用使用次数','当月飞机类应用使用次数','当月火车类应用使用次数','当月旅游资讯类应用使用次数']
predictors_continues = list(set(predictors) - set(predictors_category))

In [35]:
data['用户话费敏感度'].value_counts()

4    29838
5    21011
2    20622
3    20578
1     7913
0       38
Name: 用户话费敏感度, dtype: int64

In [13]:
class Feature:
    def __init__(self, data, predictors):
        self.name = 'feature1'
        self.data = data
#         self.predictors = predictors
        
    def single_data_distribution(self):
    # identity 3
        self.data.loc[self.data['用户年龄'] == 0, '用户年龄'] = np.nan
        #
        self.data['month'] = self.data['用户网龄（月）'] // 12
        #
        self.data['year'] = self.data['用户网龄（月）'] % 12
        
    # business 2
        self.data.loc[self.data['用户话费敏感度'] == 0, '用户话费敏感度'] = 1 # self.data['用户话费敏感度'].median()
             #
        self.data['charge_type'] = 0
        self.data.loc[(self.data['缴费用户最近一次缴费金额（元）'] % 10 == 0) & (self.data['缴费用户最近一次缴费金额（元）'] != 0), 'charge_type'] = 1
#         return self.data
    
    def single_feature_count(self):
    # business 4
        feature_count_list = ['缴费用户最近一次缴费金额（元）', '用户近6个月平均消费值（元）', '用户当月账户余额（元）', '用户账单当月总费用（元）']
        for ff in feature_count_list:
            self.data = feature_count(self.data, ff)
        
    def single_num_ceilling(self):
    # business 4 
        feature_list = ['缴费用户最近一次缴费金额（元）', '用户近6个月平均消费值（元）', '用户当月账户余额（元）', '用户账单当月总费用（元）']
        for ff in feature_list:
            new_feature_name = 'ceilling_' + ff
            self.data[new_feature_name] = self.data[ff].apply(num_ceilling)

    
    def single_frequency_coding(self):
        feature_list = ['缴费用户最近一次缴费金额（元）', '用户近6个月平均消费值（元）', '用户账单当月总费用（元）']
        for ff in feature_list:
            new_feature_name = 'fre_coding_' + ff
            self.data[new_feature_name] = self.data[ff].apply(endnumber)
    
    def composite_data_distribution(self):
    # app 6
        all_ratio_list_origin = ['当月视频播放类应用使用次数', '当月旅游资讯类应用使用次数', '近三个月月均商场出现次数']
        df_scale = preprocessing.StandardScaler().fit_transform(self.data[all_ratio_list_origin]) # 先归一化
        df_scale = pd.DataFrame(df_scale, columns=all_ratio_list_origin)
        for ff in all_ratio_list_origin:
            all_ratio_list = all_ratio_list_origin.copy()
            all_ratio_list.remove(ff)
            new_feature_name = ff + '/all'
            # 归一化
            ff_scale = df_scale[ff]
            self.data[new_feature_name] = ff_scale / df_scale[all_ratio_list].sum(axis=1)
#             print(new_feature_name)
            all_ratio_list.append(ff)
            
    def composite_business(self):
    # business 13
        # 各种比例 7
        self.data['user_6_pay_rate'] = self.data['用户近6个月平均消费值（元）'] / (self.data['缴费用户最近一次缴费金额（元）'] + 5)
        self.data['user_balance_pay_rate'] = self.data['用户当月账户余额（元）'] / (self.data['缴费用户最近一次缴费金额（元）'] +5)
        self.data['user_6_balance_rate'] = self.data['用户近6个月平均消费值（元）'] / (self.data['用户当月账户余额（元）'] + 5)
        self.data['user_balance_cost_rate'] = self.data['用户当月账户余额（元）'] / (self.data['用户账单当月总费用（元）'] + 5)
        self.data['user_6_cost_rate'] = self.data['用户近6个月平均消费值（元）'] / (self.data['用户账单当月总费用（元）'] + 5)
        self.data['user_pay_cost_rate'] = self.data['缴费用户最近一次缴费金额（元）'] / (self.data['用户账单当月总费用（元）'] + 5)
        self.data['user_cost_comparision'] = self.data['用户近6个月平均消费值（元）'] / (self.data['用户账单当月总费用（元）'] + 5)
        # 过去5个月和当月的消费情况 2
        self.data['last_five_fee'] = self.data['用户近6个月平均消费值（元）']*6 - self.data['用户账单当月总费用（元）']
        self.data['now-five'] = self.data['用户账单当月总费用（元）'] - self.data['last_five_fee']/5
        
        # 推导用户上月支出、上月支出缺失值、上月余额 3
        index_0 = self.data['用户最近一次缴费距今时长（月）']==0
        index_1 = self.data['用户最近一次缴费距今时长（月）']==1
        self.data['用户上月支出'] = np.nan
        self.data['用户上月余额'] = np.nan
        # 用户上月支出
        self.data.loc[index_1, '用户上月支出'] = self.data.loc[index_1, '缴费用户最近一次缴费金额（元）'] - self.data.loc[index_1, '用户当月账户余额（元）'] - self.data.loc[index_1, '用户账单当月总费用（元）']
        self.data.loc[index_0, '用户上月余额'] = self.data.loc[index_0,'用户当月账户余额（元）'] + self.data.loc[index_0,'用户账单当月总费用（元）'] - self.data.loc[index_0,'缴费用户最近一次缴费金额（元）']
        # 用户上月支出值缺失
        self.data.loc[index_0, '用户上月支出值缺失'] = 1
        self.data.loc[index_1, '用户上月支出值缺失'] = 0
        #  用户上月余额
        self.data.loc[index_1, '用户上月余额'] = self.data.loc[index_1, '缴费用户最近一次缴费金额（元）'] - self.data.loc[index_1, '用户上月支出']
        self.data.loc[index_0, '用户上月余额'] = self.data.loc[index_0, '用户当月账户余额（元）'] + self.data.loc[index_0, '用户账单当月总费用（元）'] - self.data.loc[index_0, '缴费用户最近一次缴费金额（元）']
        # 从 用户话费敏感度 出发 1
        self.data['余额敏感度'] = self.data.apply(lambda x: x['用户话费敏感度'] * x['用户上月余额'] if x['用户上月余额']>0 else\
                       (6 - x['用户话费敏感度']), axis=1) 
        
        # app 1
        self.data['buy_rate'] = self.data['当月物流快递类应用使用次数'] / (self.data['当月网购类应用使用次数'] + 1)
#         return self.data
    
    def drop_columns_na(self):
        self.data.drop(columns=['是否大学生客户','是否黑名单客户','当月是否到过福州山姆会员店','当月是否逛过福州仓山万达','是否经常逛商场的人','用户上月支出'], inplace=True)
        self.data.dropna(inplace=True)
        self.data.reset_index(drop=True, inplace=True)
        
    def save_data(self):
        print("输出数据的尺寸:", self.data.shape)
        self.data.to_csv("../data/all_data_fea4.csv", index=False, encoding='utf-8')

In [14]:
case = Feature(data, predictors)
case.single_data_distribution()
case.single_feature_count()
case.single_num_ceilling()
case.single_frequency_coding()
case.composite_data_distribution()
case.composite_business()
case.drop_columns_na()
case.save_data()

输出数据的尺寸: (99462, 55)
